In [5]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader

loader = TextLoader("speech.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The following keynote speech was delivered by former U.S. President Jimmy Carter on Jan. 25, 2005, as part of the inaugural Lecture Series of the Americas at the Organization of American States in Washington, D.C.  \n\nI am honored to address the Permanent Council of the Organization of American States. Thank you, Mr. Secretary General, Mr. President, and Ambassador Borea for the kind invitation to inaugurate this Lecture Series of the Americas.\n\nI have long been interested in this organization. Thirty years ago, as governor of Georgia, I invited the OAS General Assembly to meet in Atlanta–the first meeting in the U.S. outside of Washington. Later, as President, I attended and addressed every General Assembly in Washington.\n\nBack then, I realized that most of this hemisphere was ruled by military regimes or personal dictatorships. Senate hearings had just confirmed U.S. involvement in destabilizing the government of Salvado

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [12]:
# Web based loader

from langchain_community.document_loaders import WebBaseLoader
import bs4

## load, chunk and index content of the html page

loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                           class_=("post-title","post-content","post-header")
                       )))
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [23]:
## Pdf reader

from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('LLM Powered Autonomous Agents _ Lil\'Log.pdf')

docs = loader.load()
# docs

In [33]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-04-06T14:56:19+00:00', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'moddate': '2025-04-06T14:56:19+00:00', 'source': "LLM Powered Autonomous Agents _ Lil'Log.pdf", 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content='Posts Archive Search Tags FAQ\nLLM Powered Autonomous Agents\nDate: June 23, 2023 | Estimated Reading Time: 31 min | Author: Lilian Weng\nTable of Contents\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several\nproof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring\nexamples. The potentiality of LLM extends beyond generating well-written copies, stories,\nessays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomou

In [34]:
## Vector Embedding and Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
db=Chroma.from_documents(documents=documents[:10], embedding=OpenAIEmbeddings())

In [35]:
## Chroma Vector database

query="What is Subgoal and decomposition?"
result=db.similarity_search(query)
result[1].page_content

'search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or\nmajority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1." ,\n"What are the subgoals for achieving XYZ?" , (2) by using task-specific instructions; e.g.\n"Write a story outline."  for writing a novel, or (3) with human inputs.\n4/6/25, 9:56 AM LLM Powered Autonomous Agents | Lil\'Log\nhttps://lilianweng.github.io/posts/2023-06-23-agent/ 2/24'

In [36]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db2=FAISS.from_documents(documents=documents[:10], embedding=OpenAIEmbeddings())


In [37]:
query="What is Subgoal and decomposition?"
result=db2.similarity_search(query)
result[1].page_content

'execution capability, access to proprietary information sources and more.\nFig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning\nA complicated task usually involves many steps. An agent needs to know what they are and\nplan ahead.\nTask Decomposition\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for\nenhancing model performance on complex tasks. The model is instructed to “think step by\nstep” to utilize more test-time computation to decompose hard tasks into smaller and simpler\nsteps. CoT transforms big tasks into multiple manageable tasks and shed lights into an\ninterpretation of the modelʼs thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at\neach step. It first decomposes the problem into multiple thought steps and generates multiple\nthoughts per step, creating a tree structure. The search process can be BFS (breadth-first'

## Advanced RAG Concepts

In [48]:
from langchain_ollama import OllamaLLM

llm=OllamaLLM(model="llama3.2")
llm

OllamaLLM(model='llama3.2')

In [49]:
## Design ChatPrompt Template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}""")



In [50]:
## Chain 

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [51]:
## Retrievers -> Look documentation

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x128f65540>, search_kwargs={})

In [52]:
## Retrieval chain

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [54]:
response=retrieval_chain.invoke({"input": "Explain Planning component in Agent system?"})

In [55]:
response['answer']

'The planning component in the agent system involves breaking down complex tasks into smaller, manageable steps. This is achieved through two techniques:\n\n1. **Chain of Thought (CoT)**: The model is instructed to "think step by step" to utilize more test-time computation and decompose hard tasks into simpler steps. CoT transforms big tasks into multiple manageable tasks.\n\n2. **Tree of Thoughts (Yao et al. 2023)**: It extends Chain of Thought by exploring multiple reasoning possibilities at each step, generating multiple thoughts per step, and creating a tree structure. The search process can be performed using Breadth-First Search (BFS).\n\nAdditionally, the LLM+P approach involves relying on an external classical planner to perform long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem.\n\nThe planning component is crucial for autonomous agents as it enables them to understand what 